# 1. SETUP

## 1.1 Import of needed libraries

In [112]:
import numpy as np
from PIL import Image
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

## 1.2 Basic transformations to all incoming data 
RGB values scaled to values between 0 and 1, data transformed into tensor datatype

In [113]:
transform = transforms.Compose([
    transforms.ToTensor(), #transformation into tensor datatype, also converting RGB into the 0-1 range
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) #normalize with a mean of 0.5 and a standard deviation of 0.5 to get values between -1 to 1 (better range to work with)
])

## 1.3 Importing train/testdata

In [114]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True) #downloading data from the CIFAR10 dataset and labelling it as train/test data
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2) #loading the data from created directory in shuffled batches of size 32, 2 parallel workers -> faster
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## 1.4 Checking the structure of incoming data

In [115]:
image, label = train_data[0]

In [116]:
image.size() #output meaning: 3 channels (RGB), image resolution 32x32 pixels

torch.Size([3, 32, 32])

## 1.5 Defining classes of output data

In [117]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] #labels for output classes, CNN output will be an integer from 0-9 corresponding to these labels

# 2.0 Building the Convolutional Neural Network (CNN)

## 2.1 Creating the CNN class
Within the init method layers can be added and values can be changed except for the 3 input channels and the 10 output neurons/classes, note that all numbers must be compatible with previous steps and therefore calculated correctly

In [118]:
class NeuralNet(nn.Module): #base class from pytorch for a neural network
    
    def __init__(self): #constructor method for defining the structure
        super().__init__() #to be able to call __init__ method of parent class

        #convolutional layer
        self.conv1 = nn.Conv2d(3, 12, 5) #defining arguments of applied convolution: 3 for input channels RGB, 12 for the amount of the created feature maps, kernel size of 5x5 producing the feature maps
                                         #calculation of output shape: (input size - kernel size) / kernel stride + 1(?) so (32-5) / 1 + 1 = 28 leads to outputs of (12, 28, 28) or 12 28x28p feature maps
        #pooling layer
        self.pool = nn.MaxPool2d(2, 2) #reshaping the convolution output by the average value of grids of 2x2 pixels, leading to a new shape of (12, 14, 14)

        #second convolutional layer
        self.conv2 = nn.Conv2d(12, 24, 5) #creating 24 feature maps from the 12 created before with kernel size of 5x5 -> new shape: (24, 10, 10), second pooling step made in later defined forward logic

        #first dense or fully connected layer (these include weights/biases?)
        self.fc1 = nn.Linear(24 * 5 * 5, 120) #multiplication to flatten the shape, 120 neurons as outputs
        #second fc layer
        self.fc2 = nn.Linear(120, 84) #120 neurons connecting to 84 neurons
        #third fc layer
        self.fc3 = nn.Linear(84, 10) #84 neurons connecting to the amount of neurons corresponding to the desired amount output possibilities

    
    def forward(self, x): #x being the input which the forward method applies the formerly defined layers onto
        x = self.pool(F.relu(self.conv1(x))) #pooling the result of the ReLU activation function (for breaking linearity) applied to the first convolution of input x
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x)) #applying ReLU to the input passing connected layers
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## 2.2 Initializing the CNN

In [119]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss() #TODO - what exactly is a loss function doing
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) #applying Stochastic Gradiant Descent momentum optimization on params of the net, learning rate = 0.001 - TODO what is an optimizer

## 2.3 Defining epochs
Epochs are the training cycles the network repeats to learn

In [120]:
for epoch in range(31):
    print(f'Training epoch {epoch}...') #printing the current cycle

    running_loss = 0.0 #for adding up and keeping track of the loss

    for i, data in enumerate(train_loader): #getting batches from the train loader
        inputs, labels = data

        optimizer.zero_grad() #resetting the gradiants to none

        outputs = net(inputs) #feeding the inputs to the network

        loss = loss_function(outputs, labels) #feeding these calculated outputs and desired outputs/labels to the loss function
        loss.backward() #calling backpropagation on the gotten loss, already implemented in pytorch
        optimizer.step() #a 'step forward' depending on the learning rate

        running_loss += loss.item() #saving tensor values in the track keeping variable

    print(f'Loss: {running_loss / len(train_loader):.4f}') #printing out the average loss calculated above all made steps

Training epoch 0...
Loss: 2.2138
Training epoch 1...
Loss: 1.8036
Training epoch 2...
Loss: 1.5385
Training epoch 3...
Loss: 1.4076
Training epoch 4...
Loss: 1.3155
Training epoch 5...
Loss: 1.2404
Training epoch 6...
Loss: 1.1679
Training epoch 7...
Loss: 1.1067
Training epoch 8...
Loss: 1.0518
Training epoch 9...
Loss: 1.0033
Training epoch 10...
Loss: 0.9637
Training epoch 11...
Loss: 0.9215
Training epoch 12...
Loss: 0.8919
Training epoch 13...
Loss: 0.8517
Training epoch 14...
Loss: 0.8216
Training epoch 15...
Loss: 0.7854
Training epoch 16...
Loss: 0.7585
Training epoch 17...
Loss: 0.7302
Training epoch 18...
Loss: 0.7047
Training epoch 19...
Loss: 0.6812
Training epoch 20...
Loss: 0.6551
Training epoch 21...
Loss: 0.6312
Training epoch 22...
Loss: 0.6046
Training epoch 23...
Loss: 0.5845
Training epoch 24...
Loss: 0.5642
Training epoch 25...
Loss: 0.5476
Training epoch 26...
Loss: 0.5176
Training epoch 27...
Loss: 0.4987
Training epoch 28...
Loss: 0.4797
Training epoch 29...
Los

## 2.4 Saving parameters
This is just saving all important data of the trained CNN so no need to train every time before using

In [121]:
torch.save(net.state_dict(), 'trained_net.pth')

In [122]:
net = NeuralNet() #creating a new instance of neural network
net.load_state_dict(torch.load('trained_net.pth')) #loading the previously calculated data as parameters for the new net instance

C:\Users\User\AppData\Local\Temp\ipykernel_21372\384279683.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.pth')) #loading th

<All keys matched successfully>

## 2.5 Evaluation on test data

In [123]:
correct = 0 #keeping track of the correctly classified data
total = 0

net.eval() #network set to evaluation mode

with torch.no_grad(): #disabling gradiant computation since there is no training here, just feeding input
    for data in test_loader:
        images, labels = data
        outputs = net(images) #saving the output of the 32 image batches fed to the network
        _, predicted = torch.max(outputs, 1) #extracting the one value of all outputs with the highest activation, so the prediction made
        total += labels.size(0) #32 in this case due to set batch size
        correct += (predicted == labels).sum().item() #saving the sum of predicitons equal to labels, so the correct classifications made

accuracy = 100 * correct / total #calculating overall accuracy

print (f'Accuracy: {accuracy}%')

Accuracy: 68.38%


# 3.0 Testing on new images
Setup to be able to load new images not included in the dataset for actual testing with unknown data

In [125]:
new_transform = transforms.Compose([ #transformation made to input images to fit in the used format
    transforms.Resize((32, 32)), #resizing input images to 32x32 pixels
    transforms.ToTensor(), #transforming into tensor datatype
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image) #applying transformation onto given input images
    image = image.unsqueeze(0) #adding batch dimension for the input for the correct input shape (even if its only 1 image)
    return image

image_archive = [] #loading all images with .png datatype out of a folder into a list
for filename in glob.glob('./unseen_images/*.png'):
    im = load_image(filename)
    image_archive.append(im)

net.eval()
with torch.no_grad():
    for image in image_archive:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f'Prediction: {class_names[predicted.item()]}')

Prediction: car
Prediction: frog
